In [ ]:
# 三方库引入

import torch, os
import numpy as np

import random
import time
import argparse
import logging
import shutil

from copy import deepcopy
from matplotlib import pyplot as plt
from tqdm import tqdm

from utils.engine import build_dataset, build_optimizer, build_scheduler, build_criterion, build_model
from utils.util import AvgrageMeter, pearson_correlation_coefficient, update_avg_meters, cal_psd_hr
from archs.PAMNet import PAMNet


def set_seed(seed=92):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
# argparser引入
parser = argparse.ArgumentParser()
## ! general params.
parser.add_argument('--num_rppg', type=int, default=160, help='the number of rPPG')
parser.add_argument('--dataset', type=str, default='VIPL', help='dataset = [VIPL, UBFC, PURE, COHFACE]')
parser.add_argument('--dataset_dir', type=str, default='/data2/chushuyang/VIPL', help='dataset dir')
parser.add_argument('--vipl_fold', type=int, default=1, help='the fold of VIPL dataset')
parser.add_argument('--save_path', type=str, default='', help='the path to save the model [ckpt, code, visulization]')
parser.add_argument('--save_mode', type=str, default='all', help='save mode [all, best]')

## ! train params.
parser.add_argument('--gpu', type=str, default="0", help='gpu id list')
parser.add_argument('--img_size', type=int, default=128, help='the length of clip')
parser.add_argument('--batch_size', type=int, default=4, help='batch size per gpu')
parser.add_argument('--eval_step', type=int, default=1, help='the number of **epochs** to eval')
parser.add_argument('--epochs', type=int, default=200, help='the number of epochs to train')
parser.add_argument('--echo_batches', type=int, default=500, help='the number of **mini-batches** to print the loss')
### loss
parser.add_argument('--loss', type=str, default='["np_loss", "ce_loss"]', help='loss = [np_loss, ce_loss]')
parser.add_argument('--loss_weight', type=str, default='[0.1, 2]', help='loss_weight = [1, 1]')
### eval_option
parser.add_argument('--eval_type', type=str, default='video', help='eval_type = [clip, video]')

## ! model params.
parser.add_argument('--model', type=str, default='PAMNet', help='model = [Efficientphys, Physnet, Physformer, PAMNet]')
parser.add_argument('--dropout', type=float, default=0.2, help='dropout rate')
### optim
parser.add_argument('--optim', type=str, default='adam', help='optimizer = [adam, sgd]')
parser.add_argument('--lr', type=float, default=1e-4, help='learning rate')
parser.add_argument('--beta1', type=float, default=0.9, help='beta1 for adam')
parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for adam')
parser.add_argument('--weight_decay', type=float, default=5e-5, help='weight decay for optimizer')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum for sgd')
### scheduler
parser.add_argument('--scheduler', type=str, default='step', help='scheduler = [step]')
parser.add_argument('--step_size', type=int, default=50, help='learning rate decay step size')
parser.add_argument('--gamma', type=float, default=0.5, help='learning rate decay')

args = parser.parse_args([])

set_seed(92)

In [3]:
# 模型导入
rppg_estimator = build_model(args).to(f'cuda:{args.gpu}')
ckpt_dir = './ckpts/VIPL_fold_1' 
epoch_load = 20
rppg_estimator.load_state_dict(torch.load(f'{ckpt_dir}/rppg_estimator_{epoch_load}.pth', map_location=torch.device(f'cuda:{args.gpu}')))

<All keys matched successfully>

In [4]:
# 数据集导入
dataloader_test = build_dataset(args, 'val_video', 1)
print(next(iter(dataloader_test))['ecg'].shape)

torch.Size([1, 1009])


In [ ]:
# video-level -- test
bpm_range = torch.arange(40, 180, dtype=torch.float).cuda()
hr_gt = []
hr_pred = []

device = f'cuda:{args.gpu}'

with torch.no_grad():
    # num_clip = 3
    for sample_batched in tqdm(dataloader_test):
        # get the inputs
        inputs, ecg, clip_average_HR = sample_batched['video'].to(device),\
            sample_batched['ecg'].to(device), sample_batched['clip_avg_hr'].to(device)

        num_clip = 3
        input_len = inputs.shape[2]
        input_len = input_len - input_len % (num_clip * 32)
        clip_len = input_len // num_clip
        inputs = inputs[:, :, :input_len, :, :]
        ecg = ecg[:, :input_len]

        new_args = deepcopy(args)
        new_args.num_rppg = clip_len
        val_rppg_estimator = build_model(new_args).to(device)
        val_rppg_estimator.load_state_dict(torch.load(f'{ckpt_dir}/rppg_estimator_{epoch_load}.pth'))
        val_rppg_estimator.eval()
        psd_gt_total = 0
        psd_pred_total = 0
        for idx in range(num_clip):

            inputs_iter = inputs[:, :, idx*clip_len : (idx+1)*clip_len, :, :]
            ecg_iter = ecg[:, idx*clip_len : (idx+1)*clip_len]

            psd_gt = cal_psd_hr(ecg_iter, 30, return_type='psd')
            psd_gt_total += psd_gt.view(-1).max(0)[1].cpu() + 40

            ## for rppg_estimator:
            all_inputs = {
                'input_clip': inputs_iter,
                'epoch': 11
            }
            outputs = val_rppg_estimator(all_inputs)
            rPPG = outputs['rPPG']

            psd_pred = cal_psd_hr(rPPG[0], 30, return_type='psd')
            psd_pred_total += psd_pred.view(-1).max(0)[1].cpu() + 40

        hr_pred.append(psd_pred_total / num_clip)
        hr_gt.append(psd_gt_total / num_clip)


print(f'mae of model: {np.mean(np.abs(np.array(hr_gt) - np.array(hr_pred)))}')
print(f'rmse of model: {np.sqrt(np.mean(np.square(np.array(hr_gt) - np.array(hr_pred))))}')